In [1]:
 
import numpy as np

#Fetching the dataset
import pandas as pd


#import transaction data and the promos data
data_transact = pd.read_csv('transactions.csv',delimiter=',', encoding="utf-8-sig")
data_promo = pd.read_csv('promos.csv',delimiter=',', encoding="utf-8-sig")

#view top 10 rows
data_transact.head(10)


,id,history_date,item_id,price,inventory,sales,category_id
0,103,2014-01-01,394908219,24.97,374.0,NaN,3
1,223,2014-01-01,394942631,106.33,374.0,NaN,3
2,319,2014-01-01,511895631,31.98,374.0,NaN,3
3,427,2014-01-01,512317778,33.21,102.0,NaN,3
4,487,2014-01-01,512319955,117.62,139.0,NaN,3
5,571,2014-01-01,512464656,287.46,235.0,NaN,3
6,18,2014-01-01,394853569,119.04,271.0,NaN,3
7,282,2014-01-01,395334759,30.05,242.0,NaN,3
8,306,2014-01-01,395360835,121.58,0.0,NaN,3
9,402,2014-01-01,512319957,17.44,146.0,138.05,3


In [2]:
data_promo.head(10)

,item_id,promo_type,promo_start_dt,promo_end_dt
0,394848103,PROMO_C,2014-06-24,2014-07-24
1,394848314,PROMO_B,2014-04-03,2014-05-03
2,394859626,PROMO_B,2014-07-23,2014-08-22
3,394862997,PROMO_A,2015-01-09,2015-02-08
4,394867143,PROMO_C,2014-03-19,2014-04-18
5,394867160,PROMO_B,2014-04-14,2014-05-14
6,394868816,PROMO_A,2014-06-28,2014-07-28
7,394881368,PROMO_C,2016-03-21,2016-04-20
8,394883117,PROMO_A,2014-09-30,2014-10-30
9,394889903,PROMO_C,2014-09-16,2014-10-16


In [3]:
train, target = pd.DataFrame(data_transact ), pd.DataFrame(data_promo)



# Imputing missing sales data with 0 value

In [4]:
#Impute the values using scikit-learn SimpleImpute Class
from sklearn.impute import SimpleImputer
imp_zeros = SimpleImputer( missing_values=np.nan , strategy='constant', fill_value=0 ,copy=True)#fill missing values with 0 constant
imp_zeros.fit(train)
imputed_train_df = imp_zeros.transform(train)

In [5]:
imputed_train_df.shape


(400059, 7)

In [6]:
imputed_tr = pd.DataFrame(imputed_train_df, columns=train.columns)
imputed_tr.head(12)

,id,history_date,item_id,price,inventory,sales,category_id
0,103,2014-01-01,394908219,24.97,374,0,3
1,223,2014-01-01,394942631,106.33,374,0,3
2,319,2014-01-01,511895631,31.98,374,0,3
3,427,2014-01-01,512317778,33.21,102,0,3
4,487,2014-01-01,512319955,117.62,139,0,3
5,571,2014-01-01,512464656,287.46,235,0,3
6,18,2014-01-01,394853569,119.04,271,0,3
7,282,2014-01-01,395334759,30.05,242,0,3
8,306,2014-01-01,395360835,121.58,0,0,3
9,402,2014-01-01,512319957,17.44,146,138.05,3


# Negative sales are interpreted as returns and should be subtracted from any aggregations of sales.

# Units can be estimated as floor(sales / price)

In [7]:
imputed_tr['Units'] = imputed_tr['sales']/imputed_tr['price']
imputed_tr.head(15)

,id,history_date,item_id,price,inventory,sales,category_id,Units
0,103,2014-01-01,394908219,24.97,374,0,3,0
1,223,2014-01-01,394942631,106.33,374,0,3,0
2,319,2014-01-01,511895631,31.98,374,0,3,0
3,427,2014-01-01,512317778,33.21,102,0,3,0
4,487,2014-01-01,512319955,117.62,139,0,3,0
5,571,2014-01-01,512464656,287.46,235,0,3,0
6,18,2014-01-01,394853569,119.04,271,0,3,0
7,282,2014-01-01,395334759,30.05,242,0,3,0
8,306,2014-01-01,395360835,121.58,0,0,3,0
9,402,2014-01-01,512319957,17.44,146,138.05,3,7.91571


# Aggregate the data to weekly level, so that there is one row per product-week combination



In [8]:
 #Converting date to pandas datetime format
imputed_tr['history_date'] = pd.to_datetime(imputed_tr['history_date'])

imputed_tr['Week_Number'] = imputed_tr['history_date'].dt.week# Getting week number

# Getting year. Weeknum is common across years to we need to create unique index by using year and weeknum
imputed_tr['Year'] = imputed_tr['history_date'].dt.year

# Grouping based on YEAR AND week number
df2 = imputed_tr.groupby([  'Week_Number'])
df2.head(10)#visualizing 

,id,history_date,item_id,price,inventory,sales,category_id,Units,Week_Number,Year
0,103,2014-01-01,394908219,24.97,374,0,3,0,1,2014
1,223,2014-01-01,394942631,106.33,374,0,3,0,1,2014
2,319,2014-01-01,511895631,31.98,374,0,3,0,1,2014
3,427,2014-01-01,512317778,33.21,102,0,3,0,1,2014
4,487,2014-01-01,512319955,117.62,139,0,3,0,1,2014
5,571,2014-01-01,512464656,287.46,235,0,3,0,1,2014
6,18,2014-01-01,394853569,119.04,271,0,3,0,1,2014
7,282,2014-01-01,395334759,30.05,242,0,3,0,1,2014
8,306,2014-01-01,395360835,121.58,0,0,3,0,1,2014
9,402,2014-01-01,512319957,17.44,146,138.05,3,7.91571,1,2014


# Create any additional features you feel would help in modeling

# Make sure to perform any other transformations for missing values, outliers, improperly scaled information, or anything else that would represent a best practice for data scientists.